In [7]:
%load_ext autoreload
%autoreload 2
%aimport -sys
%aimport -rpy2
%aimport -pandas
%aimport -numpy
%aimport -plotly

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys

if '..' not in sys.path:
    sys.path.insert(0, '..')

import glm
import plotter
import plotly.express as px
from pandas import DataFrame
from numpy import ndarray
import pandas as pd
import numpy as np
from plotly.graph_objects import Figure as PlotlyFigure
import statsmodels.api as sm
from scipy.stats import norm
# from statsmodels.discrete.discrete_model import Logit as LogisticRegressor
from statsmodels.formula.api import logit as LogisticRegressor
from statsmodels.discrete.discrete_model import Logit
from sklearn.linear_model import LogisticRegression
from patsy import dmatrix

In [12]:
N = 10
results: Dict = glm.compare_sklearn(features=N)

dict_keys(['model', 'dataframe', 'input_matrix', 'computed_y', 'true_y', 'fitted_y', 'true_coef'])

In [13]:
plot_df = results["dataframe"].copy()
for i in range(1, N + 1):
    plot_df[f"feat_{i}_bin"] = pd.qcut(
        x=plot_df[f"feat_{i}"],
        q=10,
        # include_lowest=True,
        labels=False,
    )


In [11]:
# plot_df.corr().style.background_gradient(cmap='coolwarm')

In [12]:
boxplots = []
for i in range(1, N + 1):
    fig: PlotlyFigure = plotter.boxplot(
        data=plot_df.sample(100000), x=f"feat_{i}_bin", y="y", points=False
    )
    boxplots.append(fig)

plotter.save_plots_to_html(figures=boxplots, file="./exhibits/boxplots.html")
del boxplots


In [13]:
scatterplots = []
for i in range(1, N + 1):
    fig: PlotlyFigure = px.scatter(
        data_frame=plot_df.sample(50000), x=f"feat_{i}", y="y", render_mode="webgl"
    )
    scatterplots.append(fig)

plotter.save_plots_to_html(  # can only render 8 at most
    figures=scatterplots, file="./exhibits/scatterplots.html"
)
del scatterplots


In [14]:
for i in range(1, N+1):
    fig = px.histogram(plot_df.sample(25000), x=f'feat_{i}', histnorm='probability density')

In [16]:
plotter.qqplot(results['fitted_y'] - results['true_y'])

# Exploring Logistic Regression

In [3]:
data = sm.datasets.get_rdataset("Duncan", "carData").data
data['prestige_ge_60'] = np.where(data['prestige'] >= 60, 1, 0)
data.head()

,type,income,education,prestige,prestige_ge_60
accountant,prof,62,86,82,1
pilot,prof,72,76,83,1
architect,prof,75,92,90,1
author,prof,55,90,76,1
chemist,prof,64,86,90,1


In [4]:
predictors = ["income", "education"]
formula = "+".join(predictors)
# model: Logit = LogisticRegressor(formula=f"prestige_ge_60 ~ {formula}", data=data).fit()


In [5]:
X = dmatrix(formula, data)
sklearn_model = LogisticRegression(penalty="none").fit(
    X=X,
    y=data["prestige_ge_60"].values,
)


In [7]:
(
    model.predict(X)
    - sklearn_model.predict_proba(X=X)[:, 1]
)


# Exploring QQ Plots